# Description

Notebook for breast cancer screening RDD discovery analysis. Exercises the end-to-end process of RDD discovery.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import seaborn as sns
import sys

from tqdm import tqdm

In [ ]:
# sample split for TMLR rebuttal
from sklearn.model_selection import train_test_split

In [ ]:
# user imports 
sys.path.append("../../")

from rddd.feat import gen_feat_df, get_descriptives
#from rddd.rddd import policy_tree_discovery, test_discontinuity, create_feat_df

In [ ]:
sys.path.append("/home/liutony/optum-pipeline/notebooks/tmlr/")

import rdsgd

In [ ]:
# notebook magics
%load_ext autoreload

%autoreload 2

%matplotlib inline

# Load data

In [ ]:
%%time
bc_df = pd.read_parquet("/project/liu_optum_causal_inference/data/breast_cancer/merge/breast_cancer.parq")

In [ ]:
bc_df.columns

# Clean data

In [ ]:
%%time
bc_feat = gen_feat_df(bc_df)

In [ ]:
bc_feat.dtypes

# Run assignment tree discovery

In [ ]:
test_df = bc_feat.copy()

In [ ]:
import warnings
warnings.filterwarnings("ignore", module='sk.*')

In [ ]:
%%time
grid_dict = {
    'age': np.arange(40, 61, 5)
}
alpha = 0.05
treat = 'indicator'
running_cols = ['age']
bw = 4
tree_kwargs = {
    'max_depth': 2,
    'min_balancedness_tol': 0.3,
}

random_state = 42

# downsample for memory issues
sample_df = test_df.copy()

In [ ]:
# add in sample splitting
s1_df, s2_df = train_test_split(sample_df, test_size=0.5, random_state=random_state)

In [ ]:
%%time
subgroup_dict, num_tests = rdsgd.rd_subgroup_discovery(s1_df,
                                                 running_cols=running_cols,
                                                 grid_dict=grid_dict,
                                                 treat=treat,
                                                 alpha=alpha,
                                                 rescale=False,
                                                 omit_mask=True,
                                                 bw=bw,
                                                 #tree_kwargs=tree_kwargs,
                                                 #random_state=random_state
                                                )

In [ ]:
for cutoff, nodes in subgroup_dict['age'].items():
    for node in nodes:
        rule_path = node['rule_path']
        holdout = s2_df.copy()
        for rule in rule_path[:-1]:
            if rule.path_dir == '<':
                holdout = holdout[holdout[rule.feature] < rule.threshold]
            elif rule.path_dir == '>=':
                holdout = holdout[holdout[rule.feature] >= rule.threshold]
            elif rule.path_dir == '<=':
                holdout = holdout[holdout[rule.feature] <= rule.threshold]
            elif rule.path_dir == '>':
                holdout = holdout[holdout[rule.feature] > rule.threshold]
            elif rule.path_dir == '==':
                holdout = holdout[holdout[rule.feature] == rule.threshold]

        llr_results, _, _ = rdsgd.test_discontinuity(holdout, cutoff, 'age', treat=treat, bw=bw, kernel='triangular')
        node['llr_results'] = llr_results

In [ ]:
subgroup_dict['age'][40]

In [ ]:
out_dir = "/project/liu_optum_causal_inference/results/tmlr_sample_split/"

In [ ]:
pickle.dump((subgroup_dict, num_tests), open(f"{out_dir}/breast_cancer_subgroup_results_tmlr.pkl", "wb"), -1)

# Extract baseline and subgroup data

In [ ]:
sorted_nodes = sorted(sel_nodes, key=lambda x: x[1]['posthoc_pwr'], reverse=True)

In [ ]:
cutoff = 40
running = 'age'
bw = 4
baseline_df = create_feat_df(sample_df, running=running, 
                             cutoff=cutoff, bw=bw) 

In [ ]:
baseline_df['in_subgroup'] = (sorted_nodes[2][1]['subgroup_mask']).astype(int)

baseline_df[['age', 'indicator', 'in_subgroup']].to_parquet("/project/liu_optum_causal_inference/results/breast_cancer_running.parq")

#baseline_df[sorted_nodes[2][1]['subgroup_mask']].copy()

In [ ]:
for col in baseline_df.columns:
    get_descriptives(baseline_df, col)

In [ ]:
baseline_df['age'].describe()